# GAANN - Genetic Algorithm and Artificial Neural Network

This notebook tries to explain the details behind the implemented GAANN algorithm with an example.

All the details are readable in the code.

## The basic idea

The GAANN is the combinaison of a [genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) and an [artificial neural network](https://en.wikipedia.org/wiki/Multilayer_perceptron). It is a wrapper method since the output of the ANN, the score, is used to improve the classifier. As all the algorithms in this project, the classifier's score itself is not important but the features that give the best score are and the score is therefore used as an indicator.

Let's see how it works.

In [ ]:
def timeit(method):
    """
    Just a simple method to mesure the execution time of a method
    """
    import time

    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()

        print('%r %2.2f sec' % (method.__name__, te - ts))
        return result

    return timed

In [ ]:
# The used score function is F1-Score. This function can leads to 0/0 division.
# Theses following lines hide warnings about 0/0 divisions when computing the F-Score. 
# When looking at the source code, all 0/0 divisions are set to 0. 
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [ ]:
# increase font size in matplotlib
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

In [ ]:
from datasets.DatasetEncoder import DatasetEncoder
from datasets.DatasetSplitter import DatasetSplitter
from datasets.Golub99.GolubDataset import GolubDataset
from datasets.DatasetLoader import DatasetLoader
from datasets.DatasetBalancer import DatasetBalancer

from algorithms.GAANNAlgorithm import GAANNAlgorithm

from matplotlib import pyplot as plt
%matplotlib inline

# Load dataset from environment variable. This is used by automated scripts
ds_class = DatasetLoader.load_from_env_var(default_dataset="MILE")

print("Dataset used: %s" % ds_class.__name__)

ds = ds_class()

# encode Dataset string classes into numbers
ds_encoder = DatasetEncoder(ds)
ds = ds_encoder.encode()
ds = DatasetSplitter(ds, test_size=0.4)

ds_balancer = DatasetBalancer(ds)
ds = ds_balancer.balance()

In [ ]:
# with verbose=True, we can see what are the slowest methods.
N_FEATURES_TO_KEEP = 300
gaanaa = GAANNAlgorithm(ds, N_FEATURES_TO_KEEP, verbose=True)

# accessing to a private member is generally not a good idea, but see: https://mail.python.org/pipermail/tutor/2003-October/025932.html
ga = gaanaa._ga

## Selection

The selection method is used to select the individuals for the next generation. The used method is fitness proportionate selection (https://en.wikipedia.org/wiki/Fitness_proportionate_selection).

## Crossover method

The crossover method is about combining two individuals, called male and female in the code, to create an other individual, called child in the code.

The child creation consists of theses steps:
1. take the features shared by both parents
2. fill the remaining feature space using random unique features from the parents


## Mutation method

The mutation methods consists in attributing random unique features to a child. The mutation rate is fixed to 0.05.

## Keep the elite

To ensure that the best individuals are not lost at the next generation, they are kept before doing any genetic operations. At the moment only the best individual is kept but this is tunable.

## Fitness method (ANN)

The fitness methods is a MLP which performs on all the dataset with a crossvalidation method (3 times). The mean of the runs is taken as the fitness score. The higher the score is the better the individual is. The crossvalidation allows to stabilize the score. 

## Population score evolution

The graph below shows the evolution of the population score at each generation. The population score is computed using the 75th percentile of the individuals score. This is allow us to know that that at least 75% of the individuals (lists) in the population has a score equals or greater than the printed score. The individual score is based on F1 weigthed score method. This avoid the algorithm to focus on absolute accuracy which can be misleading in case of unbalanced dataset. See: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score

In a previous implementation, only the very best individual was shown at each generation using bigger lists (1000 features). With the current implementation (100 features and 75th percentile) nearly all the generated lists are good enough and can be combined to build a bigger list (for example 1000 like the previous implementation)

This algorithms works well with the Golub dataset since it shows that the score is increasing but using MILE dataset the score evolution is flat. Thus, the algorithm is working but it is not very helping in the case of MILE because the same score can be achieved with a random list.

In [ ]:
scores = ga._score_history
plt.figure(figsize=(10, 6))
plt.plot(scores)
plt.title("Evolution of the score \n75th percentile score of the population at each generation")
plt.ylim(0, 1.1)
plt.xlabel("Generations")
plt.ylabel("Score")
plt.grid(True)

As we can see above, the evolution of the score does not increase as much as expected.

## The best list obtained

In [ ]:
best_f = max(ga._fitness_history, key=lambda x:x[1])
print("Best score (%.2f) with this list of features by GAANN \n%s" % (best_f[1], best_f[0]))

## Evolution of the similarity between the best list at each generation

The following graph shows the evolution of the similarity between the best list at generation N and the best list at generation N+1.

With this graph, it shows that the algorithm seems to work because the lists are getting similar. Sometimes, there is a peak at y=1.0, this happens when the best lists at generation N and N+1 are exactly the same.

In [ ]:
from itertools import islice, izip_longest

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def jaccard(a, b):
    return len(a.intersection(b))/float(len(a.union(b)))

history_similarity = list()

for n, m in window(ga._fitness_history):
    features_n = set(n[0])
    features_m = set(m[0])
    history_similarity.append(jaccard(features_n, features_m))

plt.figure(figsize=(12, 4))
plt.plot(history_similarity, marker='o', linestyle='-')
plt.xlim(-1, )
plt.ylim(0, 1.1)
plt.xlabel("Generations")
plt.ylabel("Similarity")
plt.grid(True)
plt.title("Jaccard similarity between the best lists of generations N and N+1")

We can see that the curve is increasing, this means that the lists are getting similar. 

# Related works

When this algorithm was written some others works have been done but not shown in this notebook. Among these works, there is the will of parallelise the fitness function. The main idea is to run multiple MLP in parallel since the implementation of the latter in Scikit is monocore. Multiple attempts have been tried like :
1. Using `multiprocessing` python module. The expected gain was about n times where n is the number of cores available. The reality showed that the gain was about 1.5 to 2 times faster than the sequential version. This is caused by the overhead of manipulating the objects which need to be pickled because of the [GIL](https://wiki.python.org/moin/GlobalInterpreterLock).
2. Using `IPython.parallel` python module.
3. Using an other library than Scikit : Keras was chosen to see if the low-level implementation with Theanos speeds up the algorithm. In practice the benefits was not so obvious.

Finally the kept method is the sequential one (using the `map` build-in method) because the crossvalidation already parallelise the jobs with `n_jobs=-1` (all the tests were performed without the crossvalidation at the time).

Here is a comparaison table of the performance of the implementations tried on MILE dataset, 1000 features no crossvalidation.

| Python map (sequential)  – ram : 5-6 GB | Multiprocessing – ram: 9-12 GB | Iparallel – ram: 12-16 GB     | 
|-----------------------------------------|--------------------------------|-------------------------------| 
| '_create_population' 0.07 sec           | '_create_population' 0.05 sec  | '_create_population' 0.05 sec | 
| '_crossover' 0.10 sec                   | '_crossover' 0.11 sec          | '_crossover' 0.09 sec         | 
| '_mutate' 0.00 sec                      | '_mutate' 0.00 sec             | '_mutate' 0.00 sec            | 
| '_evolve' 0.10 sec                      | '_evolve' 0.11 sec             | '_evolve' 0.09 sec            | 
| '_grade_pop' 112.70 sec                 | '_grade_pop' 54.70 sec         | '_grade_pop' 51.51 sec        | 
| '_crossover' 0.12 sec                   | '_crossover' 0.13 sec          | '_crossover' 0.09 sec         | 
| '_mutate' 0.00 sec                      | '_mutate' 0.00 sec             | '_mutate' 0.00 sec            | 
| '_evolve' 0.12 sec                      | '_evolve' 0.13 sec             | '_evolve' 0.09 sec            | 
| '_grade_pop' 86.51 sec                  | '_grade_pop' 56.95 sec         | '_grade_pop' 57.31 sec        | 
| '_crossover' 0.11 sec                   | '_crossover' 0.12 sec          | '_crossover' 0.10 sec         | 
| '_mutate' 0.00 sec                      | '_mutate' 0.00 sec             | '_mutate' 0.00 sec            | 
| '_evolve' 0.11 sec                      | '_evolve' 0.12 sec             | '_evolve' 0.10 sec            | 
